In [ ]:
# Install or update openai modules 
!pip install openai python-dotenv mutagen
!pip install --upgrade openai
!pip install edge-tts
!pip install pygame
!pip install python-dotenv
!pip install pydub
!pip install beautifulsoup4

In [3]:
# Load Modules
from openai import OpenAI
import IPython
from datetime import datetime
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC, TPE1, TALB, TCON
import sys
from dotenv import load_dotenv
import os

import asyncio
import edge_tts
import pygame
from pydub import AudioSegment
from bs4 import BeautifulSoup

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
def read_lines_from_file(file_path):
  with open(file_path, 'r') as file:
    lines = file.readlines()
  return lines

# Example usage
file_path = 'idioms.txt'  # Replace with the path to your text file
lines = read_lines_from_file(file_path)
print(lines)
print(len(lines))

# # save the idioms to a file
# with open('idioms.txt', 'w') as f:
#     for item in lines:
#         f.write("%s\n" % item)

In [13]:
load_dotenv()
client = OpenAI(
)

In [ ]:
def get_base_text(idiom):
  role_definition = """
  你是我的英语教练。我将提供一个带有编号的 idiom，你要帮我把他翻译成合适的中文释疑。
  而后，讲解一下这个 idiom 的来历=，以及适用的场合。
  再然后，你要为这个 idiom 提供 3 个美式英文例句，尽量使用日常词汇，不要过长。
  
  [Overall Rules to follow]
    1. Do not compress your responses.

  参照以下例子（markdown）作为输出格式(follow the Overall Rules to follow)，
  其中，请格外注意 span 和 audio html 标签的正确，以及，弯引号外与汉字之间应该有一个空格：
  
```  
## 1. 9-to-5 

**释义**

“朝九晚五” 或 “平常的上班时间”。

这个短语起源于标准的工作时间，即从上午9点到下午5点，这也是劳动法定时计工作时间。在现代社会，“9-to-5” 已经成为了描述一个常规工作日的固定表达，用以表示这样的工作常规和一般的工作时间安排。

This phrase originates from the standard working hours, from 9 AM to 5 PM, which are also the legally defined work hours. In modern society, "9-to-5" has become a fixed expression to describe a regular workday, indicating such a working routine and general schedule.

**Examples**

- <span lang="en">She works a typical *9-to-5* job at an office.</span> <span lang="cn">她在办公室做一份典型的朝九晚五的工作。</span><audio controls><source src="audio/001.1.mp3" type="audio/mpeg"></audio>
- <span lang="en">I'm tired of the *9-to-5* grind and want a more flexible schedule.</span> <span lang="cn">我厌倦了朝九晚五的枯燥生活，想要一个更灵活的时间安排。</span><audio controls><source src="audio/001.2.mp3" type="audio/mpeg"></audio>
- <span lang="en">Not everyone is suited for a *9-to-5* routine.</span> <span lang="cn">并不是所有人都适合朝九晚五的作息。</span><audio controls><source src="audio/001.3.mp3" type="audio/mpeg"></audio>

```
"""
  
  user_prompt = idiom
  
  # how many versions needed.
  number_of_choices = 1
  
  rspd_translation = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
          "role": "system", 
           "content": role_definition
      },
      {
          "role": "user", 
          "content": user_prompt
      }
    ],
    n = number_of_choices 
  )
  return rspd_translation.choices[0].message.content

markdown_text = get_base_text("38. A penny for your thoughts")
print(markdown_text)

In [15]:
def get_example_list(text):
  soup = BeautifulSoup(text, 'html.parser')
  english_spans = soup.find_all('span', lang='en')
  return [span.get_text() for span in english_spans]

example_list = get_example_list(markdown_text)
print(example_list)

['You’ve been awfully quiet during the meeting. *A penny for your thoughts?*', 'She looked out the window, lost in thought. He said, "*A penny for your thoughts*."', '"*A penny for your thoughts,*" I asked my friend who seemed puzzled.']


In [16]:
def add_zero(n):
  if n < 10: 
    return f'00{n}'
  if n >9 and n <100: 
    return f'0{n}'
  if n >= 100: 
    return f'{n}'

print(add_zero(400))

400


In [17]:
import random
def get_example_sentence_audio(example_sentence, number, order):
  text = example_sentence
  n = add_zero(number)
  
  voice_list = ['alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer']
  voice_performer = random.choice(voice_list)
  # alloy, echo, fable, onyx, nova, and shimmer, the last two of which are femail voices.
  
  speech_file_path = f'audio/{n}.{order}.mp3'
  
  rspd_audio = client.audio.speech.create(
    model="tts-1",
    voice=voice_performer,
    input=text
  ) 
  
  rspd_audio.stream_to_file(speech_file_path)
  print(f'The audio of {n} is saved...')


# for e in example_list:
#   get_example_sentence_audio(e.replace("*", ""), 38, example_list.index(e)+1)


In [18]:
import re

def extract_number_from_string(s):
    match = re.search(r'\d+', s)
    if match:
        return int(match.group())
    else:
        return None  # Return None if no number is found

# Example usage
input_string = "44. Costs an arm and a leg"
number = extract_number_from_string(input_string)
print(number)  # Output: 44

44


In [31]:
with open("book.md", 'w') as file:

  for l in lines:
    markdown_text = get_base_text(str(lines.index(l)+1) + '. ' + l.strip())  
    print(markdown_text)
    file.write(markdown_text)
    
    example_list = get_example_list(markdown_text)
    for e in example_list:
      get_example_sentence_audio(e.replace("*", ""), lines.index(l)+1, example_list.index(e)+1)
    
    file.write("\n\n")
    
    print(l)
          
print("All done!")
  
    

## 1. 9-to-5 

**释义**

“朝九晚五” 或 “平常的上班时间”。

这个短语起源于标准的工作时间，即从上午 9 点到下午 5 点，这也是劳动法定时计工作时间。在现代社会，“9-to-5” 已经成为了描述一个常规工作日的固定表达，用以表示这样的工作常规和一般的工作时间安排。

This phrase originates from the standard working hours, from 9 AM to 5 PM, which are also the legally defined work hours. In modern society, "9-to-5" has become a fixed expression to describe a regular workday, indicating such a working routine and general schedule.

**Examples**

- <span lang="en">She works a typical *9-to-5* job at an office.</span> <span lang="cn">她在办公室做一份典型的朝九晚五的工作。</span><audio controls><source src="audio/001.1.mp3" type="audio/mpeg"></audio>
- <span lang="en">I'm tired of the *9-to-5* grind and want a more flexible schedule.</span> <span lang="cn">我厌倦了朝九晚五的枯燥生活，想要一个更灵活的时间安排。</span><audio controls><source src="audio/001.2.mp3" type="audio/mpeg"></audio>
- <span lang="en">Not everyone is suited for a *9-to-5* routine.</span> <span lang="cn">并不是所有人都适合朝九晚五的作息。</span><audio controls><source src="audio/001.3.mp3" type="audi

/var/folders/37/8tk8sqmn12n9p8bq1tlj4lqw0000gn/T/ipykernel_75948/2167635988.py:18: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  rspd_audio.stream_to_file(speech_file_path)


The audio of 001 is saved...
The audio of 001 is saved...
The audio of 001 is saved...
9-to-5

## 2. A bad apple 

**释义**

“害群之马” 或 “一个有害的个体”。

这个短语的起源可以追溯到一条古老的谚语 “一个烂苹果会把整篮苹果都弄坏”（One bad apple spoils the whole barrel）。这表明，一个负面或有害的个体可能会影响周围的人或整个群体。因此，在现代用法中，“a bad apple” 仍然用于指一个对群体有负面影响的人或事物。

This phrase originates from the old saying “one bad apple spoils the whole barrel,” illustrating that a negative or harmful individual can affect the entirety of their surroundings or group. In modern usage, "a bad apple" continues to signify someone or something that negatively impacts a group.

**Examples**

- <span lang="en">It only takes *a bad apple* to ruin the entire team dynamic.</span> <span lang="cn">只需一个害群之马就能破坏整个团队的氛围。</span><audio controls><source src="audio/002.1.mp3" type="audio/mpeg"></audio>
- <span lang="en">They had to fire him because he was *a bad apple* disrupting the workplace harmony.</span> <span lang="cn">他们不得不开除他，因为他是一个破坏职场和谐的害群之马。</span><audio controls><source src="au

In [ ]:
from pydub import AudioSegment
import os

# combine audios into larger chunck
combined = AudioSegment.empty()

for i in range(1351, 1356):
  for j in range(1,4):
    print(i, j)
    if j == 1:
      combined += AudioSegment.from_file("1-sec.mp3")
      combined += AudioSegment.from_file("Funk.mp3")
      combined += AudioSegment.from_file("1-sec.mp3")
    combined += AudioSegment.from_file(f"audio/{add_zero(i)}.{j}.mp3")
    combined += AudioSegment.from_file(f"3-sec.mp3")
    if (i % 10 == 0 or i == 1355) and j == 3:
      combined += AudioSegment.from_file("ending.mp3")
      combined.export(f"{int(i/10)-1}1-{i}.mp3")
      combined = AudioSegment.empty()

In [57]:
from pydub import AudioSegment

def create_silence_mp3(duration_seconds, output_file):
    # Create a silent audio segment
    silence = AudioSegment.silent(duration=duration_seconds * 1000)  # duration in milliseconds
    # Export the silent audio segment as an MP3 file
    silence.export(output_file, format="mp3")

# Duration of silence in seconds
n = 10  # Change this to the desired number of seconds
output_file = "1-sec.mp3"  # Output file name

create_silence_mp3(1, output_file)

print(f"{n} seconds of silence have been saved to {output_file}.")

10 seconds of silence have been saved to 1-sec.mp3.


In [74]:
def add_metadata(mp3_file, image_file, artist, album, genre):
    try:
        audio = MP3(mp3_file, ID3=ID3)

        # Add ID3 tag if it doesn't exist
        try:
            audio.add_tags()
        except:
            pass

        # Add artist name
        audio.tags.add(TPE1(encoding=3, text=artist))

        # Add album name
        audio.tags.add(TALB(encoding=3, text=album))

        # Add genre
        audio.tags.add(TCON(encoding=3, text=genre))

        # Add artwork
        with open(image_file, 'rb') as albumart:
            audio.tags.add(APIC(
                encoding=3,          # 3 is for utf-8
                mime='image/jpeg',   # image/jpeg or image/png
                type=3,              # 3 is for the cover image
                desc=u'Cover',
                data=albumart.read()
            ))

        audio.save()
        print(f"Metadata added to {mp3_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
mp3_files = [file for file in os.listdir('audio/') if file.endswith('.mp3')]
for f in mp3_files:
  image_file = 'Cover.png'
  artist = '@xiaolai & ChatGPT/TTS'
  album = 'Most Common American Idioms'
  genre = 'SPEECH'
  add_metadata(f"audio/{f}", image_file, artist, album, genre)